# Origin

In [ ]:
@staticmethod
def load_data():
    # parser = get_parser()
    # args = parser.parse_args()

    human_eval = load_dataset("humaneval_infilling", "HumanEval-MultiLineInfilling")
    # human_eval = load_dataset("humaneval_infilling", "HumanEval-SingleLineInfilling")
    # human_eval = load_dataset("humaneval_infilling", "HumanEval-RandomSpanInfilling")
    # human_eval = load_dataset("humaneval_infilling", "HumanEval-RandomSpanInfillingLight")
    # Generate completions for evaluation set
    n_tasks = len(human_eval["test"])

    prefixes = []
    suffixes = []
    solutions = []
    for task in range(n_tasks):
        prefixes.append(human_eval["test"][task]["prompt"])
        suffixes.append(human_eval["test"][task]["suffix"])
        solutions.append(human_eval["test"][task]["canonical_solution"])

    jsonl_file = 'codegen.jsonl'  # args.jsonl_file
    num_samples = 1  # args.num_samples

    print("build the LLM")
    SIZE = '350M'  # 350M 2B 6B
    MODEL_NAME = f"Salesforce/codegen-{SIZE}-mono"
    tok = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to("cuda")
    tok.pad_token = tok.eos_token

    print("build generations")
    completed_codes = list()
    for idx, (prompt, suffix) in tqdm(enumerate(zip(prompts, suffixes))):
        # prompt = prompt.replace("\n\n\n", "\n\n")
        # print('---' * 19)
        # print(f'+++{idx}', prompt)
        completed_code = infill_code(prompt, num_samples=num_samples)
        completed_codes.append(completed_code)
        print(f'###{idx}', completed_code)

    print("save generations")
    with open(jsonl_file, 'w') as file:
        for completed_code in completed_codes:
            file.write(json.dumps(completed_code) + '\n')

    TEMPLATE_TO_INFILL = """
Leading context: "{leading_context}"
Trailing context: "{trailing_context}"
Given the leading and trailing context as above, what code to infill?
Code to infill: """

TEMPLATE_TO_INFILL.format(leading_context=leading_context, trailing_context=trailing_context)

# My

In [1]:
#@title Library
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Pip
import json
import torch

from datasets import load_dataset, load_metric
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
#@title Arguments
jsonl_file = 'codegen.jsonl'  # args.jsonl_file
num_samples = 1  # args.num_samples

dataset = "loubnabnl/humaneval_infilling"
subset = "HumanEval-MultiLineInfilling" #@param ["HumanEval-MultiLineInfilling", "HumanEval-SingleLineInfilling", "HumanEval-RandomSpanInfilling", "HumanEval-RandomSpanInfillingLight"]
specified_data_size = 1 #@param {allow-input: true}

model_name = "bigcode/santacoder"

In [ ]:
#@title Data
human_eval = load_dataset(dataset, subset)

# Generate completions for evaluation set
prefixes = []
suffixes = []
solutions = []

data_size = min(specified_data_size, len(human_eval["test"]))
for task in range(data_size):
    prefixes.append(human_eval["test"][task]["prompt"])
    suffixes.append(human_eval["test"][task]["suffix"])
    solutions.append(human_eval["test"][task]["canonical_solution"])

In [5]:
#@title Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, revision="fim", trust_remote_code=True)
if torch.cuda.is_available():
  model.to("cuda")
tokenizer.pad_token = tokenizer.eos_token

In [6]:
input_text = "<fim-prefix>def print_hello_world():\n    <fim-suffix>\n    print('Hello world!')<fim-middle>"
inputs = tokenizer.encode(input_text, return_tensors="pt")
outputs = model.generate(inputs, max_length=100, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<fim-prefix>def print_hello_world():
    <fim-suffix>
    print('Hello world!')<fim-middle>print('Hello world!')

if __name__ == '__main__':<|endoftext|><fim-prefix><fim-suffix>e.log(err);
    }
}

module.exports = {
    get: get,
    post: post
}<fim-middle>const axios = require('axios');

const get = async (url) => {
    try {
        const response = await axios.get(url);
        return response.


In [ ]:
#@title Generation
completed_codes = list()
for idx, (prefix, suffix) in tqdm(enumerate(zip(prefixes, suffixes))):
  completed_code = model.generate(prefix)
  completed_codes.append(completed_code)
  print(f'###{idx}', completed_code)

In [ ]:
prefixes